In [133]:
# load in libraries

import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [135]:
# import csv file to filter sources by country 

country_filter = pd.read_csv('/Users/KevinLubin/Desktop/ds/pearl/gdelt_country_sources.csv')
country_filter.head()

,source,fips
0,0-100.it,IT
1,0-50.ru,RS
2,0-60mag.com,US
3,0-debt.com,US
4,000fff.org,US


In [64]:
# define credentials object for GCP to run queries

credentials = service_account.Credentials.from_service_account_file(
    '/Users/KevinLubin/Desktop/ds/pearl/pearl-336700-0fa91569420d.json')

In [136]:
# Perform query

query = """

    SELECT GlobalEventID, SQLDATE, EventRootCode, Actor1CountryCode, Actor2CountryCode, Quadclass, AvgTone,
    GoldSteinScale, NumMentions, Sourceurl

    FROM `gdelt-bq.full.events`
    
    WHERE (SQLDATE >= 20210101 AND SQLDATE <= 20210105)
    
    AND ActionGeo_CountryCode = 'HA'
    
    GROUP BY SQLDATE, GlobalEventID, EventRootCode, Actor1CountryCode, Actor2CountryCode, Quadclass, AvgTone, 
    GoldSteinScale, NumMentions, Sourceurl
    
    ORDER BY SQLDATE, GlobalEventID, EventRootCode, Actor1CountryCode, Actor2CountryCode, Quadclass, AvgTone,
    GoldSteinScale, NumMentions, Sourceurl
    
"""

df = pandas_gbq.read_gbq(query, credentials=credentials)

Downloading: 100%|██████████| 261/261 [00:00<00:00, 1025.13rows/s]


In [142]:
df.head()

,GlobalEventID,SQLDATE,EventRootCode,Actor1CountryCode,Actor2CountryCode,Quadclass,AvgTone,GoldSteinScale,NumMentions,Sourceurl
0,962251396,20210101,07,HTI,USA,2,-2.325581,7.0,2,https://www.bizjournals.com/baltimore/news/202...
1,962254970,20210101,17,None,None,4,-3.227166,-5.0,4,https://www.timesnownews.com/international/art...
2,962254971,20210101,17,None,None,4,-3.227166,-5.0,16,https://www.timesnownews.com/international/art...
3,962255205,20210101,18,HTI,PAK,4,-3.227166,-9.0,4,https://www.timesnownews.com/international/art...
4,962280685,20210101,04,BRA,ESP,1,-0.504202,1.0,6,https://dissidentvoice.org/2021/01/canadas-all...


In [125]:
df.Sourceurl.to_list()

['https://www.havanatimes.org/?p=135074',
 'https://www.boston25news.com/news/rwandan-camera-kids-bring-photography-legacy-to-boston-foster-kids/793643924',
 'https://www.hltv.org/forums/threads/1837709/who-are-minorities-in-your-country',
 'https://www.hltv.org/forums/threads/1837709/who-are-minorities-in-your-country',
 'http://fourwinds10.com/siterun_data/government/foreign_policy_and_government/news.php?q=1540222554',
 'https://msutoday.msu.edu/news/2018/four-star-general-former-un-executive-and-retired-congresswoman-to-address-graduates/',
 'https://www.vnews.com/George-Sykes-Applies-for-Open-Council-Seat-27587206',
 'https://www.vnews.com/George-Sykes-Applies-for-Open-Council-Seat-27587206',
 'https://www.vnews.com/George-Sykes-Applies-for-Open-Council-Seat-27587206',
 'https://www.vnews.com/George-Sykes-Applies-for-Open-Council-Seat-27587206',
 'https://www.pri.org/stories/2019-09-09/lessons-un-peacekeeping-mission-rwanda-25-years-after-genocide-it-failed-stop',
 'https://www.si

In [139]:
# filtering to sources from lebanon

lebanon_sources = country_filter[country_filter.fips == "LE"] 

In [143]:
lebanon_sources.head()

,source,fips
682,14march.org,LE
1515,3akarat.net,LE
2702,98weeks.net,LE
2910,aaco.org,LE
5110,afedonline.org,LE


In [141]:
lebanon_sources.shape

(124, 2)

In [ ]:
# TODO: write code to look for the base URL from above in the Sourceurl column from the query df 